## Google colab setup

In [1]:
try:
  import google.colab
  IS_COLAB = True
except:
  IS_COLAB = False

In [2]:
if IS_COLAB:
  !pip install tensorflow_addons
  !pip install unidecode
  !pip install transformers
  # !pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 24.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 13.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 130.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.8 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp /content/drive/MyDrive/SemiSupervised/*.csv .

## Imports and setup

In [5]:
%%capture
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import re
import time
import spacy
import numpy as np
from transformers import AutoTokenizer, TFAutoModel
from unidecode import unidecode
# from utils import prepare_train_ds, prepare_ds

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')

## Prepare data splits

In [6]:
data = pd.read_csv("train_ner.csv")

In [7]:
def preprocess(x):
    s = unidecode(x)
    s = str.lower(s)
    s = re.sub(r"\[[a-z]+\]","", s)
    s = re.sub(r"\*","", s)
    s = re.sub(r"[^a-zA-Z0-9]+"," ",s)
    s = re.sub(r" +"," ",s)
    s = re.sub(r"(.)\1+",r"\1",s)

    return s

In [8]:
label_ids = {label_name:i for i, label_name in enumerate(sorted(set(data["label"])))}
label_ids

{'ABUSE': 0, 'INSULT': 1, 'OTHER': 2, 'PROFANITY': 3}

In [9]:
tok_robert = AutoTokenizer.from_pretrained("readerbench/RoBERT-base")

In [10]:
def split_ssl_data(ids_array,mask_array,labels,num_classes,label_percent):
  labeled = None
  unlabeled = None

  for class_idx in range(num_classes):
    class_ids = ids_array[labels==class_idx]
    class_mask = mask_array[labels==class_idx]
    sz = int(label_percent * class_ids.shape[0])

    labels_reduced = labels[labels==class_idx][:sz]
    labeled_ids, unlabeled_ids = class_ids[:sz], class_ids[sz:]
    labeled_mask, unlabeled_mask = class_mask[:sz], class_mask[sz:]

    if not labeled:
      labeled = (labeled_ids, labeled_mask, labels_reduced)
      unlabeled = (unlabeled_ids, unlabeled_mask)
    else:
      labeled = (
          np.concatenate([labeled[0],labeled_ids]),
          np.concatenate([labeled[1],labeled_mask]),
          np.concatenate([labeled[2],labels_reduced])
      )
      unlabeled = (
          np.concatenate([unlabeled[0],unlabeled_ids]),
          np.concatenate([unlabeled[1],unlabeled_mask]),
      )

  return labeled, unlabeled

In [11]:
def preprocess_robert(x):
  t = tok_robert(x,padding="max_length",max_length=96,truncation=True,return_tensors='np')
  return t["input_ids"], t["attention_mask"]

def map_func(input_ids, masks, labels):
  return {'input_ids': input_ids, 'attention_mask': masks}, labels

def map_func2(input_ids, masks):
  return {'input_ids': input_ids, 'attention_mask': masks}

def prepare_ds(filename,batch_size=64,label_percent=0.8):
  df = pd.read_csv(filename)
  X_id_mask = df['text'].map(preprocess).apply(preprocess_robert).apply(pd.Series)

  X_id_mask.columns = ["input_ids","attention_mask"]

  ids_array = np.squeeze(np.stack(X_id_mask.input_ids.values), axis=1)
  mask_array = np.squeeze(np.stack(X_id_mask.attention_mask.values), axis=1)
  labels = df["label"].map(lambda x: label_ids[x]).values

  res_ds = tf.data.Dataset.from_tensor_slices((ids_array, mask_array, labels)).map(map_func).shuffle(len(df)).batch(batch_size)
  return res_ds

In [12]:
def prepare_train_ds(filename,batch_size=32,label_percent=0.6):
  df = pd.read_csv(filename)
  df = df.sample(frac=1)
  X_id_mask = df['text'].map(preprocess).apply(preprocess_robert).apply(pd.Series)

  X_id_mask.columns = ["input_ids","attention_mask"]

  ids_array = np.squeeze(np.stack(X_id_mask.input_ids.values), axis=1)
  mask_array = np.squeeze(np.stack(X_id_mask.attention_mask.values), axis=1)
  labels = df["label"].map(lambda x: label_ids[x]).values

  labeled, unlabeled = split_ssl_data(ids_array,mask_array,labels,len(label_ids),label_percent)

  labeled_ds = tf.data.Dataset.from_tensor_slices(labeled)
  labeled_ds = labeled_ds.map(map_func).shuffle(len(labeled_ds)).batch(batch_size).repeat()
  unlabeled_ds = tf.data.Dataset.from_tensor_slices(unlabeled)
  unlabeled_ds = unlabeled_ds.map(map_func2).shuffle(len(unlabeled_ds)).batch(batch_size).repeat()
  
  return labeled_ds, unlabeled_ds

In [13]:
labeled_ds, unlabeled_ds = prepare_train_ds("train_ner.csv")
test_ds = prepare_ds("test_ner.csv")
val_ds = prepare_ds("validation_internal_ner.csv")

## Model definition and declaration

In [14]:
class FreeMatchTune(tf.keras.Model):
  def __init__(
      self,
      encoder_name="readerbench/RoBERT-base",
      num_classes=4,
      **kwargs
  ):
    super(FreeMatchTune,self).__init__(**kwargs)

    self.bert = TFAutoModel.from_pretrained(encoder_name)
    self.num_classes = num_classes
    self.weak_augment = tf.keras.layers.GaussianNoise(stddev=0.5)
    self.strong_augment = tf.keras.layers.GaussianNoise(stddev=5)

    self.cls_head = tf.keras.Sequential([
      tf.keras.layers.Dense(256,activation="relu"),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(64,activation="relu"),
      tf.keras.layers.Dense(self.num_classes, activation="softmax")
    ])

  def call(self, inputs, training):
    ids, mask = inputs
    
    embeds = self.bert(input_ids=ids, attention_mask=mask,training=training).pooler_output

    strongs = self.strong_augment(embeds,training=training)
    weaks = self.weak_augment(embeds,training=training)

    strong_preds = self.cls_head(strongs,training=training)
    weak_preds = self.cls_head(weaks,training=training)

    return weak_preds, strong_preds
  
class FreeMatch(tf.keras.Model):
  def __init__(
      self,
      encoder_name="readerbench/RoBERT-base",
      num_classes=4,
      encoder_train=False,
      **kwargs
  ):
    super(FreeMatch,self).__init__(**kwargs)

    self.bert = TFAutoModel.from_pretrained(encoder_name)
    self.bert.trainable = encoder_train
    self.num_classes = num_classes
    self.weak_augment = tf.keras.layers.GaussianNoise(stddev=0.5)
    self.strong_augment = tf.keras.layers.GaussianNoise(stddev=5)


    self.cls_head = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(10),
            merge_mode="ave"
        ),
        tf.keras.layers.Dense(units=self.num_classes,activation="softmax")
    ])

  def call(self, inputs, training):
    ids, mask = inputs
    
    embeds = self.bert(input_ids=ids, attention_mask=mask).last_hidden_state

    strongs = self.strong_augment(embeds,training=training)
    weaks = self.weak_augment(embeds,training=training)

    strong_preds = self.cls_head(strongs)
    weak_preds = self.cls_head(weaks)

    return weak_preds, strong_preds



In [15]:
# model = FreeMatch()
model = FreeMatchTune()

## Train and evaluate

In [16]:
optim = tfa.optimizers.AdamW(weight_decay=0.001,learning_rate=0.005)
optim2 = tfa.optimizers.AdamW(weight_decay=0.0,learning_rate=0.00001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_prec_metric = tf.keras.metrics.Precision(name="precision")
val_recall_metric = tf.keras.metrics.Recall(name="recall")
f1_metric_micro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='micro', name='f1_micro')
f1_metric_macro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='macro', name='f1_macro')

In [17]:
unsup_weight = 1.0
num_classes = 4
fairness_weight = 0.01
eps = 1e-7
steps_per_epoch = 190

In [18]:
tau = tf.Variable(tf.constant(1/num_classes))
p_thresh = tf.Variable(tau * tf.ones((num_classes,),dtype=tf.float32))
hist_t = tf.Variable(eps * tf.ones((num_classes,),dtype=tf.float32))
eps_t = tf.Variable(eps * tf.ones((num_classes,),dtype=tf.float32))

ema_decay = tf.constant(0.999)

In [19]:
@tf.function
def sumnorm(x):
  return x/tf.reduce_sum(x)

@tf.function
def maxnorm(x):
  return x/tf.reduce_max(x)


@tf.function
def train_step(x,y,xu,ema_decay):

  with tf.GradientTape() as tape:
      wl,_ = model([x["input_ids"],x["attention_mask"]],training=True)
      wu,su = model([xu["input_ids"],xu["attention_mask"]],training=True)

      ls = loss_fn(y, wl)

      local_thresh_update = tf.reduce_mean(tf.reduce_max(wu,axis=1))
      tau.assign(ema_decay * tau + (1-ema_decay) * local_thresh_update)
      p_thresh.assign(ema_decay * p_thresh + (1-ema_decay) * tf.reduce_mean(wu,axis=0))

      hist_t.assign(ema_decay * hist_t + (1-ema_decay) * tf.reduce_sum(tf.one_hot(tf.argmax(wu,axis=1),4),axis=0))
      sat = tau * maxnorm(p_thresh)
      mask = tf.reduce_max(wu,axis=1)>=tf.gather(sat,tf.argmax(wu,axis=1))
      
      su = su[mask]
      wu = wu[mask]
      
      lu = loss_fn(tf.argmax(wu,axis=1),su)
      
      p_avg = tf.reduce_mean(su,axis=0)
      h_avg = tf.reduce_sum(tf.one_hot(tf.argmax(su,axis=1),4),axis=0)

      lf = -tf.keras.metrics.categorical_crossentropy(sumnorm(p_thresh/hist_t),sumnorm(p_avg/(eps_t+h_avg)))

      loss = ls + unsup_weight * lu + fairness_weight * lf

  grads = tape.gradient(loss, model.trainable_weights)
  optim.apply_gradients(zip(grads, model.trainable_weights))

  return loss

@tf.function
def train_step_tune(x,y,xu,ema_decay,uw,fw):

  with tf.GradientTape() as tape:
      wl,_ = model([x["input_ids"],x["attention_mask"]],training=True)
      wu,su = model([xu["input_ids"],xu["attention_mask"]],training=True)

      ls = loss_fn(y, wl)

      local_thresh_update = tf.reduce_mean(tf.reduce_max(wu,axis=1))
      tau.assign(ema_decay * tau + (1-ema_decay) * local_thresh_update)
      p_thresh.assign(ema_decay * p_thresh + (1-ema_decay) * tf.reduce_mean(wu,axis=0))

      hist_t.assign(ema_decay * hist_t + (1-ema_decay) * tf.reduce_sum(tf.one_hot(tf.argmax(wu,axis=1),4),axis=0))
      sat = tau * maxnorm(p_thresh)
      mask = tf.reduce_max(wu,axis=1)>=0.9 #tf.gather(sat,tf.argmax(wu,axis=1))
      
      su = su[mask]
      wu = wu[mask]
      
      lu = loss_fn(tf.argmax(wu,axis=1),su)
      
      lf = 0.0
      if tf.shape(su)[0] != 0:
        p_avg = tf.reduce_mean(su,axis=0)
        h_avg = tf.reduce_sum(tf.one_hot(tf.argmax(su,axis=1),4),axis=0)

        lf = -tf.keras.metrics.categorical_crossentropy(sumnorm(p_thresh/hist_t),sumnorm(p_avg/(eps_t+h_avg)))

      loss = ls + uw * lu + fw * lf

  grads = tape.gradient(loss, [model.cls_head.trainable_weights, model.bert.trainable_weights])
  optim.apply_gradients(zip(grads[0], model.cls_head.trainable_weights))
  optim2.apply_gradients(zip(grads[1], model.bert.trainable_weights))

  return loss

@tf.function
def test_step(x, y):
    wpred,_ = model([x["input_ids"],x["attention_mask"]], training=False)
    val_acc_metric.update_state(y, wpred)
    true_hot = tf.one_hot(y, 4)
    val_prec_metric.update_state(true_hot, wpred)
    val_recall_metric.update_state(true_hot, wpred)
    f1_metric_micro.update_state(true_hot, wpred)
    f1_metric_macro.update_state(true_hot, wpred)

In [20]:
EPOCHS=15
max_val_acc = 0.0
best_weights = None

l_iter = iter(labeled_ds)
u_iter = iter(unlabeled_ds)
for epoch in range(EPOCHS):
  print(f"Epoch {epoch}")
  start_time = time.time()
  if epoch < 2:
      unsup_weight = 0.0
      fairness_weight = 0.0
  else:
      unsup_weight = 1.0
      fairness_weight= 0.01
  uw = tf.Variable(unsup_weight)
  fw = tf.Variable(fairness_weight)

  for step in range(steps_per_epoch):
    x, y = next(l_iter)
    xu = next(u_iter)
    
    loss = train_step_tune(x,y,xu,ema_decay,uw,fw)
    if step % 50 == 0:
      print(
        "Training loss (for one batch) at step %d: %.4f"
        % (step, float(loss))
      )
  for x_batch_val, y_batch_val in val_ds:
      test_step(x_batch_val, y_batch_val)

  acc = float(val_acc_metric.result())
  prec = float(val_prec_metric.result())
  recall = float(val_recall_metric.result())
  micro = float(f1_metric_micro.result())
  macro = float(f1_metric_macro.result())

  val_acc_metric.reset_states()
  val_prec_metric.reset_states()
  val_recall_metric.reset_states()
  f1_metric_micro.reset_states()
  f1_metric_macro.reset_states()

  if acc > max_val_acc:
    max_val_acc = acc
    best_weights = model.get_weights()
  print(f"Validation acc: {acc:.3f} precision: {prec:.3f} recall: {recall:.3f} f1_micro: {micro:.3f} f1_macro: {macro:.3f}")
  print("Time taken: %.2fs" % (time.time() - start_time))
print("Restoring best weights relative to validation accuracy...")
model.set_weights(best_weights)

Epoch 0
Training loss (for one batch) at step 0: 1.5595
Training loss (for one batch) at step 50: 1.2031
Training loss (for one batch) at step 100: 1.1604
Training loss (for one batch) at step 150: 1.1311
Validation acc: 0.642 precision: 0.782 recall: 0.500 f1_micro: 0.610 f1_macro: 0.437
Time taken: 129.67s
Epoch 1
Training loss (for one batch) at step 0: 0.9377
Training loss (for one batch) at step 50: 0.7333
Training loss (for one batch) at step 100: 0.7877
Training loss (for one batch) at step 150: 1.1804
Validation acc: 0.806 precision: 0.848 recall: 0.765 f1_micro: 0.804 f1_macro: 0.796
Time taken: 73.99s
Epoch 2
Training loss (for one batch) at step 0: 1.3018
Training loss (for one batch) at step 50: 1.5276
Training loss (for one batch) at step 100: 0.9102
Training loss (for one batch) at step 150: 1.9673
Validation acc: 0.811 precision: 0.830 recall: 0.796 f1_micro: 0.813 f1_macro: 0.804
Time taken: 73.94s
Epoch 3
Training loss (for one batch) at step 0: 1.0557
Training loss (f

In [21]:
for x_batch_val, y_batch_val in test_ds:
    test_step(x_batch_val, y_batch_val)
acc = float(val_acc_metric.result())
prec = float(val_prec_metric.result())
recall = float(val_recall_metric.result())
micro = float(f1_metric_micro.result())
macro = float(f1_metric_macro.result())

val_acc_metric.reset_states()
val_prec_metric.reset_states()
val_recall_metric.reset_states()
f1_metric_micro.reset_states()
f1_metric_macro.reset_states()
print(f"Test acc: {acc:.3f} precision: {prec:.3f} recall: {recall:.3f} f1_micro: {micro:.3f} f1_macro: {macro:.3f}")

Test acc: 0.796 precision: 0.800 recall: 0.791 f1_micro: 0.795 f1_macro: 0.789


In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
model.bert.push_to_hub("andrei-saceleanu/ro-offense-freematch")

tf_model.h5:   0%|          | 0.00/461M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
model.cls_head.save_weights("./checkpoints/freematch_tune")